# Lab 3 Consultar la Metadata de una Instancia EC2 con IMDSv2 usando Instance Connect

---

**Objetivo:**

Este laboratorio te guiará en el uso de la Instance Metadata Service Version 2 (IMDSv2) para obtener información de la instancia EC2. En lugar de conectarte por SSH desde tu máquina local, usarás **Instance Connect** directamente desde la consola de AWS. Además, verás cómo cambiar entre IMDSv2 y IMDSv1 desde la consola de AWS.

**Requisitos:**

- Una instancia EC2 en ejecución que soporte Instance Connect (Amazon Linux 2 o similar).
- Permisos para acceder a la consola de AWS EC2.
- La instancia debe tener IMDSv2 habilitado (requerir tokens).

---

**Pasos del Laboratorio**

**1. Verificar la Configuración Actual de la Metadata**

1. Ve a la consola de AWS: https://console.aws.amazon.com/
2. Navega al servicio EC2.
3. Selecciona tu instancia.
4. Haz clic en **Actions > Instance Settings > Modify Instance Metadata Options**.
5. Verifica que **IMDSv2** (requiere tokens) esté habilitado:
    - "Metadata version": "IMDSv2"
    - Endpoint enabled: sí
6. Haz clic en **Save** si realizas cambios.

**2. Conectarte a la Instancia con Instance Connect**

1. En la consola de EC2, selecciona tu instancia.
2. Haz clic en **Connect** (botón superior derecho).
3. Selecciona la pestaña **EC2 Instance Connect**.
4. Haz clic en **Connect**.

Se abrirá una terminal en tu navegador conectada a la instancia.

**3. Obtener el Token de IMDSv2**

En IMDSv2, primero necesitas solicitar un token con un tiempo de vida.

1. En la terminal (Instance Connect), ejecuta:
    
    ```bash
    
    TOKEN=$(curl -X PUT "http://169.254.169.254/latest/api/token" -H "X-aws-ec2-metadata-token-ttl-seconds: 21600")
    
    ```
    
2. Verifica que obtuviste el token:
    
    ```bash
    
    echo $TOKEN
    
    ```
    

Deberías ver una cadena larga que representa el token.

**4. Consultar la Metadata Usando el Token**

Ahora que tienes el token, puedes hacer solicitudes a la metadata.

1. Listar las categorías disponibles:
    
    ```bash
    
    curl -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data/
    
    ```
    
    Verás una lista de ítems como `ami-id`, `instance-id`, `instance-type`, etc.
    
2. Obtener el ID de la instancia:
    
    ```bash
    
    curl -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data/instance-id
    
    ```
    
    El comando retornará algo como `i-1234567890abcdef0`.
    
3. Obtener el tipo de instancia:
    
    ```bash
    
    curl -H "X-aws-ec2-metadata-token: $TOKEN" http://169.254.169.254/latest/meta-data/instance-type
    
    ```
    
    Devolverá, por ejemplo, `t2.micro`.
    

Recuerda que con IMDSv2, cada vez que consultes metadata, debes usar el token. Si el token expira, genera uno nuevo (repite el paso 3).

**5. (Opcional) Cambiar a IMDSv1**

Si deseas volver a IMDSv1 (no recomendado en producción por seguridad):

1. Ve a la consola de AWS EC2.
2. Selecciona tu instancia.
3. Actions > Instance Settings > Modify Instance Metadata Options.
4. Cambia "Metadata version" para permitir IMDSv1.
5. Guarda los cambios.

Con IMDSv1, no necesitas token:

```bash

curl http://169.254.169.254/latest/meta-data/

```

Recuerda, el token nos agrega una capa adicional de seguridad!!

---

**Validación**

- Sin token, intenta acceder a la metadata:
    
    ```bash
    
    curl http://169.254.169.254/latest/meta-data/instance-id
    
    ```
    
    Debería fallar o darte un error, ya que IMDSv2 requiere token.
    
- Con el token, obtén la metadata exitosamente, confirmando que IMDSv2 está funcionando.

---

**Conclusión**

En este lab aprendiste a:

- Confirmar que la instancia usa IMDSv2.
- Obtener un token para acceder a la metadata.
- Consultar la metadata (ID de instancia, tipo de instancia, etc.) usando el token.
- Conocer cómo regresar a IMDSv1 (opcional).

El uso de IMDSv2 añade una capa de seguridad adicional, ya que requiere un token para acceder a la metadata, evitando exposiciones no autorizadas. Además, utilizaste Instance Connect para conectarte a la instancia directamente desde la consola, sin configurar claves SSH en tu equipo local.